 ## Exercise 48 - GroupBy

In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

inputPath = "/data/students/bigdata-01QYD/ex_data/Ex48/data"
outputPathV1 = "res_out_Ex48V1/"
outputPathV2 = "res_out_Ex48V2/"

__input__ : 
* A CSV file containing a list of user profiles : _name, age, gender_

__output__ : 
*  Select the names occurring at least two times and store in the output folder name and average(age) of the selected names

### DataFrames

In [4]:
# map into (movieid , userid)
df = spark.read.load(inputPath, format="csv", header=True, inferSchema=True)
df.show()

+-----+---+------+
| name|age|gender|
+-----+---+------+
| Paul| 40|  male|
| Paul| 38|  male|
|David| 15|  male|
|Susan| 40|female|
|Susan| 34|female|
+-----+---+------+



In [7]:
df_grouped = df.groupBy("name").agg({"name": "count", "age": "avg"})
df_grouped.show()

+-----+-----------+--------+
| name|count(name)|avg(age)|
+-----+-----------+--------+
|Susan|          2|    37.0|
|David|          1|    15.0|
| Paul|          2|    39.0|
+-----+-----------+--------+



In [9]:
df_grouped_filtered = df_grouped.filter("count(name) >= 2").select("name", "avg(age)")
df_grouped_filtered.show()

+-----+--------+
| name|avg(age)|
+-----+--------+
|Susan|    37.0|
| Paul|    39.0|
+-----+--------+



In [10]:
df_grouped_filtered.write.csv(outputPathV1, header=True)

### SQL

In [15]:
df_sql = spark.read.load(inputPath, format="csv", header=True, inferSchema=True)
df_sql.createOrReplaceTempView("people")

df_sql_updated = spark.sql("""  SELECT name, avg(age) as avg_age
                                FROM people
                                GROUP BY name
                                HAVING count(*)>=2
                            """)

In [16]:
df_sql_updated.show()

+-----+-------+
| name|avg_age|
+-----+-------+
|Susan|   37.0|
| Paul|   39.0|
+-----+-------+



In [17]:
df_sql_updated.write.csv(outputPathV2, header=False)